In [1]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [2]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data/balanced/"
ampl.read(mod_path)

# Assign set data

In [3]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [4]:
intervals = pd.read_csv(os.path.join(dataDirectory, "intervals.csv"),skipinitialspace=True)
dffr = pd.read_csv(os.path.join(dataDirectory, "dffr.csv"),skipinitialspace=True)
da = pd.read_csv(os.path.join(dataDirectory, "da.csv"),skipinitialspace=True)

In [5]:
assign_set_data("INTERVALS",intervals.values[:,0])
assign_set_data("DFFR_PRICE",dffr.DFFR_PRICE.values)
assign_set_data("DA_PRICE",da.DA_PRICE.unique())

# Assign parameter data

In [6]:
ampl.getParameter('Cost').set(0)
ampl.getParameter('Ramp').set(999999)
ampl.getParameter('Ramp_DFFR').set(9999999)
ampl.getParameter('P_MAX').set(2)

In [7]:
n_DFFR_PRICE = len(dffr.DFFR_PRICE.unique())

df = pd.DataFrame([1/n_DFFR_PRICE for i in range(n_DFFR_PRICE)],
             columns=["p_R"],
             index = dffr.DFFR_PRICE.values)
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

n_DA_PRICE = len(da.DA_PRICE.unique())
df = pd.DataFrame([1/n_DA_PRICE for i in range(n_DA_PRICE)],
             columns=["p_DA"],
             index = da.DA_PRICE.unique())
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

n_INTERVAL = len(intervals)

In [8]:
df = dffr.set_index("DFFR_PRICE")
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

df = da.set_index(["DA_PRICE","INTERVALS"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)


# Solve the model

In [9]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
knitro_options = 'outlev = 1 mip_outlevel = 1'
solver = 'cplexamp'
settings = {'solver': solver, 'knitro_options':knitro_options,'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [10]:
ampl.solve()

CPLEX 12.5.1.0: mipdisplay = 2
mipgap = .005
Found incumbent of value 0.000000 after 0.00 sec. (0.03 ticks)
MIP Presolve eliminated 72 rows and 96 columns.
Reduced MIP has 202 rows, 178 columns, and 454 nonzeros.
Reduced MIP has 75 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.06 ticks)
Reduced MIP has 202 rows, 178 columns, and 454 nonzeros.
Reduced MIP has 75 binaries, 0 generals, 0 SOSs, and 0 indicators.
Probing time = 0.00 sec. (0.06 ticks)
Clique table members: 25.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (0.53 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                            0.0000     3699.0100       63     --- 
*     0+    0                          725.2550     3699.0100       63  410.03%

In [11]:
## Extract solution ##
d_R  = ampl.getVariable('d_R' ).getValues().toPandas()
d_DA = ampl.getVariable('d_DA').getValues().toPandas()
q_R  = ampl.getVariable('q_R' ).getValues().toPandas()
q_DA = ampl.getVariable('q_DA').getValues().toPandas()
Q_R  = ampl.getVariable('Q_R' ).getValues().toPandas()
Q_DA = ampl.getVariable('Q_DA').getValues().toPandas()

# The level and amount of bit for each market. Be aware of the 0-based indexes.
bid_level_R   = np.where(d_R['d_R.val']==1)[0][0]           # 0-BASED INDEX.
bid_amount_R  = np.sum(q_R)
bid_level_DA  = np.zeros((n_DFFR_PRICE, n_INTERVAL), int)   # 0-BASED INDEX.
bid_amount_DA = np.zeros((n_DFFR_PRICE, n_INTERVAL))

for i in d_DA.index:
    bid_amount = q_DA.at[i, 'q_DA.val']
    if bid_amount > 1e-5:
        bid_level_DA[ int(i[0]-1), int(i[2]-1)] = int(i[1]-1)
        bid_amount_DA[int(i[0]-1), int(i[2]-1)] = bid_amount

In [12]:
print('\n- DFFR')
print('bid level: %2d, bid amount: %6.2f' % (bid_level_R+1, bid_amount_R))
                                                       # └ 0-BASED INDEX!!

print('\n- DA')
print('%6s %6s %11s %11s' % ('DFFR', 'time', 'bid level', 'bid amount'))
for i in range(n_DFFR_PRICE):
    for t in range(n_INTERVAL):
        print('%6d %6d %11d %11.2f' %
              (i+1, t+1, bid_level_DA[i, t]+1, bid_amount_DA[i, t]))
              #  └────┴─────────────────────┴── 0-BASED INDEX!!


- DFFR
bid level:  3, bid amount:   2.00

- DA
  DFFR   time   bid level  bid amount
     1      1           1        2.00
     1      2           1        2.00
     1      3           1        2.00
     1      4           1        2.00
     1      5           1        2.00
     1      6           1        2.00
     2      1           1        2.00
     2      2           1        2.00
     2      3           1        2.00
     2      4           1        2.00
     2      5           1        2.00
     2      6           1        2.00
     3      1           1        0.00
     3      2           1        0.00
     3      3           1        0.00
     3      4           1        0.00
     3      5           1        0.00
     3      6           1        0.00
     4      1           1        0.00
     4      2           1        0.00
     4      3           1        0.00
     4      4           1        0.00
     4      5           1        0.00
     4      6           1        0.00


In [13]:
# # For debug. Print when the bids are accepted.
# print('\n- Accepted bid in DFFR')
# print(Q_R.loc[Q_R['Q_R.val'] >= 1e-5])
#
# print('\n- Accepted bid in DA:')
# print(Q_DA.loc[Q_DA['Q_DA.val'] >= 1e-5])